In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import OneCycleLR, PolynomialLR

import lightning as L
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.loggers import WandbLogger

import wandb


In [2]:
df = pd.read_csv("hubble_dataset.csv")

In [3]:
df

,z,H(z),Error,Ref.
0,0.071,69.00,19.70,[44]
1,0.090,69.00,12.00,[47]
2,0.120,68.60,26.20,[44]
3,0.170,83.00,8.00,[50]
4,0.179,75.00,4.00,[48]
5,0.199,75.00,5.00,[48]
6,0.200,72.90,29.60,[44]
7,0.240,79.69,2.65,[45]
8,0.270,77.00,14.00,[50]
9,0.280,88.80,36.60,[44]


In [4]:
redshift=df[df.columns[0]].to_numpy()
Hz=df[df.columns[1]].to_numpy()
err=df[df.columns[2]].to_numpy()

In [5]:
rho0 = 1.26
rho0_b = 0.05

In [6]:
class ZData(Dataset):
    def __init__(self, z, Hz, err):
        self.z = torch.tensor(z, dtype=torch.float32).reshape(-1,1)
        self.Hz = torch.tensor(Hz, dtype=torch.float32).reshape(-1,1)
        self.err = torch.tensor(err, dtype=torch.float32).reshape(-1,1)

    def __len__(self):
        return self.z.shape[0]

    def __getitem__(self, idx):
        return self.z[idx], self.Hz[idx], self.err[idx]

In [7]:
ds = ZData(redshift, Hz, err)

In [8]:
ds[:10]

(tensor([[0.0710],
         [0.0900],
         [0.1200],
         [0.1700],
         [0.1790],
         [0.1990],
         [0.2000],
         [0.2400],
         [0.2700],
         [0.2800]]),
 tensor([[69.0000],
         [69.0000],
         [68.6000],
         [83.0000],
         [75.0000],
         [75.0000],
         [72.9000],
         [79.6900],
         [77.0000],
         [88.8000]]),
 tensor([[19.7000],
         [12.0000],
         [26.2000],
         [ 8.0000],
         [ 4.0000],
         [ 5.0000],
         [29.6000],
         [ 2.6500],
         [14.0000],
         [36.6000]]))

In [9]:
class MLP(L.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.lr          = hparams['learning_rate']
        self.batch       = hparams['batch_size']
        self.hidden_size = hparams['hidden_size']
        self.num_layers  = hparams['num_layers']
        self.epochs      = hparams['epochs']

        self.save_hyperparameters()

        H0    = torch.tensor(hparams['H0_init'])
        omg_d = torch.tensor(hparams['omg_d_init'])
        eta   = torch.tensor(hparams['eta_init'])
        B     = torch.tensor(hparams['B_init'])
        n     = torch.tensor(hparams['n_init'])

        self.H0_s    = nn.Parameter(H0, requires_grad=True)
        self.omg_d_s = nn.Parameter(omg_d, requires_grad=True)
        self.eta_s   = nn.Parameter(eta, requires_grad=True)
        self.B_s     = nn.Parameter(B, requires_grad=True)
        self.n_s     = nn.Parameter(n, requires_grad=True)

    def compute_params(self):
        self.H0    = 10 * (F.elu(self.H0_s) + 1)
        self.omg_d = F.sigmoid(0.01 * self.omg_d_s)
        self.eta   = F.sigmoid(self.eta_s)
        self.B     = F.elu(self.B_s) + 1
        self.n     = self.n_s
        
    def forward(self, x):
        c = 1.59 - 2 * self.B * (1 - self.eta) / (3 * self.n + 2 * (1 - self.eta))
        first = (1 - self.omg_d) / (1 + x)**(-3)
        second = self.omg_d / (rho0 * (1 + x) ** (3*self.n - 3))
        third  = torch.sqrt(
            2 * self.B * (1 - self.eta) * (1 + x)**(-3*self.n - 6) 
                / (3 * self.n + 2 * (1 - self.eta)) + c
        )
        return self.H0 * torch.sqrt(first + second * third)

    def training_step(self, batch, batch_idx):
        z, Hz, err = batch
        self.compute_params()
        H = self(z)
        loss = torch.sum((Hz - H)**2 / err**2)
        # loss = F.mse_loss(Hz, H)
        self.log('train_loss', loss)
        self.log('val_loss', loss)
        self.log('H0', self.H0)
        self.log('omg_d', self.omg_d)
        self.log('eta', self.eta)
        self.log('B', self.B)
        self.log('n', self.n)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": OneCycleLR(
                    optimizer,
                    max_lr = self.lr,
                    epochs = self.epochs,
                    steps_per_epoch = 1,
                ),
                "interval": "step",
                "monitor": "val_loss",
                "strict": True,
            }
        }

In [10]:
hparams = {
    'learning_rate': 1e-1,
    'batch_size': 45,
    'epochs': 100000,
    'hidden_size': 32,
    'num_layers': 5,
    'H0_init': torch.rand(1),
    'omg_d_init': torch.rand(1),
    'eta_init': torch.rand(1),
    'B_init': torch.rand(1),
    'n_init': torch.rand(1)
}

In [11]:
dl = DataLoader(ds, batch_size = 45, shuffle=True)

In [12]:
model = MLP(hparams)
model

/tmp/ipykernel_87133/2837498977.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  H0    = torch.tensor(hparams['H0_init'])
/tmp/ipykernel_87133/2837498977.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  omg_d = torch.tensor(hparams['omg_d_init'])
/tmp/ipykernel_87133/2837498977.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  eta   = torch.tensor(hparams['eta_init'])
/tmp/ipykernel_87133/2837498977.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone(

MLP()

In [13]:
wandb_logger = WandbLogger(
    project="Hubble_Predict"  
)

In [14]:
trainer = Trainer(
    logger = wandb_logger,
    max_epochs = hparams['epochs'],
    devices = [0],
    accelerator='auto',
    enable_progress_bar=False,
    callbacks=[
        LearningRateMonitor(logging_interval='epoch')
    ]
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(model, dl)

You are using a CUDA device ('NVIDIA GeForce RTX 3050 4GB Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: axect. Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type | Params
--------------------------------------
  | other params | n/a  | 5     
--------------------------------------
5         Trainable params
0         Non-trainable params
5         Total params
0.000     Total estimated model params size (MB)
/home/xteca/.local/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/xteca/.local/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
`Trainer.fit` stopped: `max_epochs=100000` reached.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f2eb1810210>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f2eb0251b10, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7f2eb0133f90, raw_cell="trainer.fit(model, dl)" store_history=True silent=False shell_futures=True cell_id=153e6a9f-d2ed-4cf2-8624-60a9e668fae1> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given

In [16]:
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f2eb1810210>> (for pre_run_cell), with arguments args (<ExecutionInfo object at 7f2e60ab3c50, raw_cell="wandb.finish()" store_history=True silent=False shell_futures=True cell_id=6169ff00-d37e-400c-a041-0de3fd84c849>,),kwargs {}:


TypeError: _WandbInit._resume_backend() takes 1 positional argument but 2 were given

B,▁▁▁▁▁▁▁▅▇▇▇████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
H0,▁▄▅▆▇▇▇▇████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eta,▅▅▅▇██▇▁▁▁▂▄▃▃▄▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
lr-Adam,▁▁▂▂▃▄▅▆▆▇███████▇▇▇▇▆▆▆▅▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁
n,▁▆▇▇███▇▆▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄
omg_d,▁▁▂▃▄▅▅▅▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇███████████████
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
B,1.76572


In [17]:
redshift

array([0.071, 0.09 , 0.12 , 0.17 , 0.179, 0.199, 0.2  , 0.24 , 0.27 ,
       0.28 , 0.352, 0.38 , 0.4  , 0.401, 0.425, 0.43 , 0.44 , 0.449,
       0.47 , 0.478, 0.48 , 0.51 , 0.52 , 0.56 , 0.57 , 0.593, 0.6  ,
       0.61 , 0.64 , 0.679, 0.73 , 0.781, 0.875, 0.88 , 0.9  , 1.037,
       1.3  , 1.363, 1.43 , 1.53 , 1.75 , 1.965, 2.3  , 2.34 , 2.36 ])